MODEL HALAL STOCK

In [1]:
from cassandra.cluster import Cluster
import pandas as pd
import yfinance as yf
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

Connect to Data base for import Data

In [2]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect('info_stock')

In [3]:
rows = session.execute('SELECT * FROM financial_data')
df = pd.DataFrame(rows.all())

In [4]:
df.head(3)

,ticker,industry,interest_income,long_business_summary,name,sector,total_assets,total_debt,total_revenue
0,SIAM.BK,"Furnishings, Fixtures & Appliances",-0.02879,Siam Steel International Public Company Limite...,SIAM_SIAM STEEL INTERNATIONAL,Consumer Cyclical,1.0,2.040780e+08,1.773048e+09
1,KAMART.BK,Household & Personal Products,0.27538,"Karmarts Public Company Limited, together with...",KAMART_KARMARTS,Consumer Defensive,1.0,6.533160e+08,3.322751e+09
2,IFS.BK,Credit Services,0.40719,IFS Capital (Thailand) Public Company Limited ...,IFS_IFS CAPITAL (THAILAND),Financial Services,1.0,2.237478e+09,4.514385e+08


In [5]:
df.shape

(921, 9)

In [6]:
df.columns

Index(['ticker', 'industry', 'interest_income', 'long_business_summary',
       'name', 'sector', 'total_assets', 'total_debt', 'total_revenue'],
      dtype='object')

Vocab about haram

In [7]:
# ✅ Vocab ที่คุณให้ไว้
vocab = {
    "Interest": {
        "Finance": [
            "interest rate", "compound interest", "loan", "mortgage", "investment"
        ],
        "Emotion": [
            "curiosity", "fascination", "passion", "hobby", "engagement"
        ]
    },
    "Gambling": [
        "bet", "wager", "casino", "odds", "jackpot", "addiction",
        "high roller", "roulette", "poker", "blackjack", "slot machine"
    ],
    "Pigs": [
        "piglet", "sow", "boar", "snout", "trough", "oink", "sty", "swine"
    ],
    "Music": [
        "melody", "harmony", "rhythm", "genre", "instrument", "lyrics",
        "composer", "band", "orchestra", "choir"
    ],
    "Prostitution": [
        "sex work", "sex worker", "brothel", "escort", "solicit", "pimp",
        "client", "john", "red-light district"
    ]
}

In [8]:

def contains_haram_keywords(text, vocab):
    text = text.lower()
    for items in vocab.values():
        if isinstance(items, dict):
            for sublist in items.values():
                if any(word in text for word in sublist):
                    return True
        else:
            if any(word in text for word in items):
                return True
    return False


In [9]:

df['has_haram_vocab'] = df['long_business_summary'].apply(lambda x: contains_haram_keywords(x, vocab))
df['halal_status'] = df.apply(lambda row: 'haram' if row['interest_income'] > 0 or row['has_haram_vocab'] else 'halal', axis=1)


In [10]:

class NumericFeatureExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X):
        return np.array(X).reshape(-1, 1)


In [11]:

features = ['long_business_summary', 'has_haram_vocab']
X = df[features]
y = df['halal_status']

preprocessor = ColumnTransformer([
    ('summary_tfidf', TfidfVectorizer(), 'long_business_summary'),
    ('haram_flag', NumericFeatureExtractor(), 'has_haram_vocab'),
    
])

model = make_pipeline(preprocessor, LogisticRegression())
model.fit(X, y)


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('summary_tfidf',
                                                  TfidfVectorizer(),
                                                  'long_business_summary'),
                                                 ('haram_flag',
                                                  NumericFeatureExtractor(),
                                                  'has_haram_vocab')])),
                ('logisticregression', LogisticRegression())])

In [12]:

y_pred = model.predict(X)
print(classification_report(y, y_pred))


              precision    recall  f1-score   support

       halal       1.00      0.03      0.06       133
       haram       0.86      1.00      0.92       788

    accuracy                           0.86       921
   macro avg       0.93      0.52      0.49       921
weighted avg       0.88      0.86      0.80       921



In [13]:
df.head(10)

,ticker,industry,interest_income,long_business_summary,name,sector,total_assets,total_debt,total_revenue,has_haram_vocab,halal_status
0,SIAM.BK,"Furnishings, Fixtures & Appliances",-0.02879,Siam Steel International Public Company Limite...,SIAM_SIAM STEEL INTERNATIONAL,Consumer Cyclical,1.0,2.040780e+08,1.773048e+09,False,halal
1,KAMART.BK,Household & Personal Products,0.27538,"Karmarts Public Company Limited, together with...",KAMART_KARMARTS,Consumer Defensive,1.0,6.533160e+08,3.322751e+09,True,haram
2,IFS.BK,Credit Services,0.40719,IFS Capital (Thailand) Public Company Limited ...,IFS_IFS CAPITAL (THAILAND),Financial Services,1.0,2.237478e+09,4.514385e+08,False,haram
3,POPF.BK,,0.00000,Prime Office Leasehold Property Fund specializ...,POPF_PRIME OFFICE LEASEHOLD,,1.0,1.908280e+08,8.941708e+08,True,haram
4,PYLON.BK,Engineering & Construction,0.08830,"Pylon Public Company Limited, together with it...",PYLON_PYLON,Industrials,1.0,6.043544e+07,9.071023e+08,False,haram
5,SYMC.BK,Telecom Services,0.31773,Symphony Communication Public Company Limited ...,SYMC_SYMPHONY COMMUNICATION,Communication Services,1.0,7.980990e+08,2.063614e+09,True,haram
6,WHAIR.BK,REIT - Industrial,0.00000,,WHAIR_WHA INDUSTRIAL LEASEHOLD,Real Estate,1.0,4.756804e+09,9.475482e+08,False,halal
7,THG.BK,Medical Care Facilities,0.06545,Thonburi Healthcare Group Public Company Limit...,THG_THONBURI HEALTHCARE GROUP,Healthcare,1.0,9.971878e+09,9.374201e+09,False,haram
8,DCON.BK,Building Materials,0.10741,"Dcon Products Public Company Limited, together...",DCON_DCON PRODUCTS,Basic Materials,1.0,9.037028e+08,1.022423e+09,False,haram
9,TAN.BK,Luxury Goods,0.11814,Tanachira Retail Corporation Public Company Li...,TAN_TANACHIRA RETAIL CORPORATIO,Consumer Cyclical,1.0,8.159490e+08,1.797406e+09,True,haram


In [17]:
import yfinance as yf
import pandas as pd

# ✅ ฟังก์ชันตรวจคำฮารอม
def contains_haram_keywords(text, vocab):
    text = text.lower()
    for items in vocab.values():
        if isinstance(items, dict):
            for sublist in items.values():
                if any(word in text for word in sublist):
                    return True
        else:
            if any(word in text for word in items):
                return True
    return False

# ✅ ฟังก์ชันหลัก: ใส่ชื่อหุ้น → ตรวจว่า Halal หรือ Haram
def check_halal_by_ticker(ticker, vocab, model, debt_ratio=0.0):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        summary = info.get("longBusinessSummary", "")

        if not summary:
            return f"⚠️ ไม่พบข้อมูล summary สำหรับ {ticker}"

        has_haram = contains_haram_keywords(summary, vocab)

        input_df = pd.DataFrame([{
            'long_business_summary': summary,
            'has_haram_vocab': has_haram,
            'debt_ratio': debt_ratio
        }])

        result = model.predict(input_df)[0]
        return f"✅ หุ้น '{ticker}' เป็น '{result.upper()}'"
    
    except Exception as e:
        return f"❌ เกิดข้อผิดพลาด: {e}"


In [18]:
print(check_halal_by_ticker("AAPL", vocab, model))      # Apple
print(check_halal_by_ticker("BBL.BK", vocab, model))    # Bangkok Bank
print(check_halal_by_ticker("PTT.BK", vocab, model))    # PTT


✅ หุ้น 'AAPL' เป็น 'HARAM'
✅ หุ้น 'BBL.BK' เป็น 'HARAM'
✅ หุ้น 'PTT.BK' เป็น 'HARAM'
